In [1]:
!pip install transformers
#!pip install langchain==0.0.191
!pip install llama-cpp-python==0.1.78
!pip install sentence-transformers
!pip install huggingface_hub
!pip install auto-gptq==0.2.2
!pip install termcolor
!pip install langchain
!pip install prompt-toolkit
!pip install InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=296593 sha256=52a11b78f84aa8ca291188e4a75e775a79538f296f4fcc4ef63c9bf7840f15c1
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=deaea6127e92a0fe91bf47603d84b2be7df84ab63ab55a8f3010652f4bc8dc1e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e74

In [2]:
from torch import torch
import InstructorEmbedding
import transformers

import os
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, LlamaForCausalLM, LlamaTokenizer, LongformerTokenizer, pipeline
import re
import shutil
import subprocess
import requests
from pathlib import Path
from auto_gptq import AutoGPTQForCausalLM
from huggingface_hub import hf_hub_download
#from InstructorEmbedding
#import InstructorEmbedding
from langchain import HuggingFacePipeline
from langchain.llms import LlamaCpp
#from llama_cpp import all
import prompt_toolkit
#from prompt_toolkit import PromptTemplate
#from llm_chain import LLMChain
#from transformers_auto_tokenizer import AutoTokenizer
#from transformers_auto_model import AutoModelForCausalLM
#from transformers_generation_config import GenerationConfig
#from transformers_llm_model import LlamaForCausalLM
#from transformers_llm_tokenizer import LlamaTokenizer
#from transformers_longformer_tokenizer import LongformerTokenizer
#from transformers_pipeline import pipeline
#from rouge import Rouge
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [3]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

def fetch_and_save_wiki_text(title):
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()

    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    return wiki_text

def clean_text(text):
    # Remove special characters except "."
    text = re.sub(r'[^A-Za-z0-9\s.\(\)\[\]\{\}]+', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

def count_tokens(text):
    tokens = tokenizer.encode(text, add_special_tokens=True)
    return len(tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [18]:
import pandas as pd

# Replace 'example.csv' with the actual name of your CSV file
csv_file_path = '/content/output_DDS.csv'

# Read the CSV file into a DataFrame using 'latin-1' encoding
df = pd.read_csv(csv_file_path, encoding='latin-1')

# Now 'df' contains your CSV data as a DataFrame
df.head()

,id,Sentence,cleaned_information,token_count
0,22,1. Click the dropdown caret\n2. Hover over an ...,1. click the dropdown caret 2. hover over an e...,1282
1,23,"Type: <b>Bug</b>\n- Choose ""View"" -> ""Appearan...",type bbugb choose view appearance tab bar sing...,1257
2,24,<!-- ???????????? Do Not Delete This! bug_repo...,do not delete this bugreporttemplate please re...,2339
3,25,Does this issue occur when all extensions are ...,does this issue occur when all extensions are ...,1246
4,26,<!-- ???????????? Do Not Delete This! bug_repo...,do not delete this bugreporttemplate please re...,1323


In [5]:
df["cleaned_information"] = df["Sentence"].apply(clean_text)
df["token_count"] = df["cleaned_information"].apply(count_tokens)
df
df.to_csv('/content/output_DDS.csv', index=False)

In [ ]:
df

In [4]:
def load_model(device_type, model_id, model_basename=None):

    print(f"Loading Model: {model_id}, on: {device_type}")
    print("This action can take a few minutes!")

    if model_basename is not None:
        if ".ggml" in model_basename:
            #logging.info("Using Llamacpp for GGML quantized models")
            model_path = hf_hub_download(repo_id=model_id, filename=model_basename)
            max_ctx_size = 4096
            kwargs = {
                "model_path": model_path,
                "n_ctx": max_ctx_size,
                "max_tokens": max_ctx_size,
            }
            if device_type.lower() == "mps":
                kwargs["n_gpu_layers"] = 1000
            if device_type.lower() == "cuda":
                kwargs["n_gpu_layers"] = 1000
                kwargs["n_batch"] = max_ctx_size
            return LlamaCpp(**kwargs)

        else:
            #logging.info("Using AutoGPTQForCausalLM for quantized models")

            if ".safetensors" in model_basename:
                # Remove the ".safetensors" ending if present
                model_basename = model_basename.replace(".safetensors", "")

            tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
            #logging.info("Tokenizer loaded")

            model = AutoGPTQForCausalLM.from_quantized(
                model_id,
                model_basename=model_basename,
                use_safetensors=True,
                trust_remote_code=True,
                device="cuda:0",
                use_triton=False,
                quantize_config=None,
            )
    elif (
        device_type.lower() == "cuda"
    ):
        #logging.info("Using AutoModelForCausalLM for full models")
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        #logging.info("Tokenizer loaded")

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True,
            trust_remote_code=True,
            # max_memory={0: "15GB"} # Uncomment this line with you encounter CUDA out of memory errors
        )
        model.tie_weights()
    else:
        #logging.info("Using LlamaTokenizer")
        tokenizer = LlamaTokenizer.from_pretrained(model_id)
        model = LlamaForCausalLM.from_pretrained(model_id)

    generation_config = GenerationConfig.from_pretrained(model_id)

    # Create a pipeline for text generation
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=2048,
        temperature=0,
        top_p=0.95,
        repetition_penalty=1.15,
        generation_config=generation_config,
    )

    local_llm = HuggingFacePipeline(pipeline=pipe)
    print("Local LLM Loaded")

    return local_llm

In [5]:
DEVICE_TYPE = "cuda" if torch.cuda.is_available() else "cpu"
SHOW_SOURCES = True
#logging.info(f"Running on: {DEVICE_TYPE}")
#ogging.info(f"Display Source Documents set to: {SHOW_SOURCES}")
model_id = "TheBloke/Llama-2-7B-Chat-GGML"
model_basename = "llama-2-7b-chat.ggmlv3.q4_0.bin"

In [6]:
LLM = load_model(device_type=DEVICE_TYPE, model_id=model_id, model_basename=model_basename)

Loading Model: TheBloke/Llama-2-7B-Chat-GGML, on: cpu
This action can take a few minutes!


llama-2-7b-chat.ggmlv3.q4_0.bin:   0%|          | 0.00/3.79G [00:00<?, ?B/s]

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


In [7]:
def generate_summary(text_chunk):
    # Defining the template to generate summary
    template = """
    Write concise summary for this text with important information adding environment and reproduction steps details to the end of summary:
    ```{text}```
    SUMMARY:
    """
    prompt = PromptTemplate(template=template, input_variables=["text"])
    llm_chain = LLMChain(prompt=prompt, llm=LLM)

    summary = llm_chain.run(text_chunk)
    return summary

In [8]:
import tqdm
!pip install textsplitter
from langchain.prompts import PromptTemplate
from tqdm import tqdm
from langchain import LLMChain
import langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
df.head()

,id,Sentence,cleaned_information,token_count
0,1,"On following video, you can see iterm2 select ...",on following video you can see iterm2 select t...,646
1,2,"[""### Discussed in https://github.com/microsof...",[ discussed in httpsgithub.commicrosoftvscodep...,1611
2,3,Type: <b>Bug</b>\nWhen I apply commit the post...,type bbugb when i apply commit the post commit...,1826
3,4,Type: <b>Bug</b>\nCC: @isidorn\nAs of version ...,type bbugb cc isidorn as of version 1.70 some ...,1335
4,5,Issue Type: <b>Bug</b>\nWhen having Python int...,issue type bbugb when having python interactiv...,1757


In [ ]:
output_dir = '/content/dds_summaries'
file_path = '/content/output_DDS.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(file_path)

for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating Summaries"):
      wonder_city = row["id"]
      text_ = row["cleaned_information"]
      summary = generate_summary(text_)
      output_txt_path = os.path.join(output_dir, f'{index + 1}.txt')
      with open(output_txt_path, 'w') as txt_file:
              txt_file.write(summary)

      print(f"Text from row {index + 1} has been saved to {output_txt_path}.")

In [9]:
text = '''Issue Type: <b>Bug</b>
How do you go to definition from peek definition window?
Before the 1.38.0 update you simply pressed enter in "peek definition" (ALT+12) and it jumped to definition.
After the update this doesn\'t work anymore?
VS Code version: Code 1.38.0 (3db7e09f3b61f915d03bbfa58e258d6eee843f35, 2019-09-03T21:49:13.459Z)
OS version: Windows_NT x64 10.0.18362
<details>
<summary>System Info</summary>
|Item|Value|
|---|---|
|CPUs|Intel(R) Core(TM) i7-4790 CPU @ 3.60GHz (8 x 3592)|
|GPU Status|2d_canvas: enabled
flash_3d: enabled
flash_stage3d: enabled
flash_stage3d_baseline: enabled
gpu_compositing: enabled
multiple_raster_threads: enabled_on
native_gpu_memory_buffers: disabled_software
oop_rasterization: disabled_off
protected_video_decode: unavailable_off
rasterization: enabled
skia_deferred_display_list: disabled_off
skia_renderer: disabled_off
surface_synchronization: enabled_on
video_decode: enabled
viz_display_compositor: disabled_off
webgl: enabled
webgl2: enabled|
|Load (avg)|undefined|
|Memory (System)|15.91GB (7.74GB free)|
|Process Argv||
|Screen Reader|no|
|VM|0%|
</details><details><summary>Extensions (4)</summary>
Extension|Author (truncated)|Version
---|---|---
Bookmarks|ale|10.5.0
python|ms-|2019.9.34474
text-marker|ryu|1.11.0
plsql-language|xyz|1.8.2
</details>
<!-- generated by issue reporter -->
(Experimental duplicate detection)
Thanks for submitting this issue.
Please also check if it is already covered by an existing one, like:
- [Peek Definition menu option missing in the "Peek Definition" window (#27128)](https://www.github.com/microsoft/vscode/issues/27128) <!-- score: 0.753 -->
- [peek definition not closed (#46065)](https://www.github.com/microsoft/vscode/issues/46065) <!-- score: 0.687 -->
- [Go to definition (#75978)](https://www.github.com/microsoft/vscode/issues/75978) <!-- score: 0.685-->
- [Go to definition (#75977)](https://www.github.com/microsoft/vscode/issues/75977) <!-- score: 0.685-->
- [Peek definition works like Go to Definition if it runing from command palette (#20952)](https://www.github.com/microsoft/vscode/issues/20952) <!-- score: 0.678 -->
<!-- potential_duplicates_comment -->
@bpasero @joaomoreno Did anything change in the tree or the workbrench tree?
It seems that the item that got initially focus cannot be selected with `Enter` anymore
The item already is both focused and selected, so pressing Enter won't fire a selection change event since the selection doesn't change (that was a tree change from some months ago, so it explains why this broke).
I copied the same `if` statement which is located 10 lines above.
Right now both events are being listened to, I believe for your use case we can simply drop the selection change and simply listen to the open event.
Hm, the weird thing is that it works in 1.37
Yeah that's pretty weird.
🤔
- VSCode Version: 1.38.0
- OS Version: ubuntu 16.04 LTS
I am using ruby extension to have intellisense and go to defintions
When there are multiple definitions for a method, while clicking on go to definition and choosing the file from the mini popup that appears, it doesn't work if I choose the definition in the same file.
It works fine if I choose definitions given in other files.
Does this issue occur when all extensions are disabled?: Can't reproduce since go to definition works on ruby extension
(Experimental duplicate detection)
Thanks for submitting this issue.
Please also check if it is already covered by an existing one, like:
- [Go to definition (#75978)](https://www.github.com/microsoft/vscode/issues/75978) <!-- score: 0.755-->
- [Go to definition (#75977)](https://www.github.com/microsoft/vscode/issues/75977) <!-- score: 0.755-->
- [how to override the default definition (#76231)](https://www.github.com/microsoft/vscode/issues/76231) <!-- score: 0.617 -->
- [Peek Definition + ENTER doesn\' go to definition anymore (#80362)](https://www.github.com/microsoft/vscode/issues/80362) <!-- score: 0.545 -->
- [Disable "go to definition" in title bar if there is no definition provider?
(#55443)](https://www.github.com/microsoft/vscode/issues/55443) <!-- score: 0.54 -->
<!-- potential_duplicates_comment -->
/duplicate of https://github.com/microsoft/vscode/issues/80362
Thanks for creating this issue! We figured it's covering the same as another one we already have.
Thus, we closed this one as a duplicate.
You can search for existing issues [here](https://aka.ms/vscodeissuesearch).
See also our [issue reporting](https://aka.ms/vscodeissuereporting) guidelines.
Happy Coding!
https://github.com/Microsoft/vscode/commit/64980ea1f3f532c82bb6c28d27bba9ef2c5b4463 debug: better cancelation support
https://github.com/Microsoft/vscode/commit/aa0231b08be315e1f58c8d133f77f9927d5c4957 update native-watchdog (https://github.com/microsoft/vscode/issues/80845)
https://github.com/Microsoft/vscode/commit/9b0b5968ad18afe511bde480b6dfb04e48ebc377 Merge pull request https://github.com/microsoft/vscode/issues/80502 from dgozman/fix-79198
https://github.com/Microsoft/vscode/commit/4f39995139df5acabcd8d51a0afb96856b78d815 Introduce `CompressibleObjectTree` (https://github.com/microsoft/vscode/issues/77876)
https://github.com/Microsoft/vscode/commit/c05a0af7dba62a49eee4bb34b6f666f68a9812a5 web - prevent unload dialog to show up from workspaces service
https://github.com/Microsoft/vscode/commit/cf88c7f3108cef1632eb603c4b40cd3837cc16cf retry command exec when having arguments, when contributes, and not yet activated, https://github.com/microsoft/vscode/issues/80338
https://github.com/Microsoft/vscode/commit/73a128e8673f425ea9d49e4cf665b1c0c86cbbe4 web - add workspace editing service support
https://github.com/Microsoft/vscode/commit/f53f3b5cc11e4aca31e2b6d5f15386b7bb625485 web - move out no-op workspace stats service
https://github.com/Microsoft/vscode/commit/126e49379d01e8c912de89d6800e0d84c71fe137 web - extract no-op tunnel service
https://github.com/Microsoft/vscode/commit/2226734db0722fa8a532af94c17c0f3e3e46fd8c :up: distro
https://github.com/Microsoft/vscode/commit/c46560d371702e8d6ab063e46d254b0b8afeed54 Merge pull request https://github.com/microsoft/vscode/issues/80839 from microsoft/tyriar/79425
https://github.com/Microsoft/vscode/commit/e27a8a0f6d93311eacc4cb89b993d45b512861e7 Merge pull request https://github.com/microsoft/vscode/issues/80738 from microsoft/tyriar/relink_automation
https://github.com/Microsoft/vscode/commit/39794796a1b49130143080a53a9dd6b2dfa3c2ad Merge pull request https://github.com/microsoft/vscode/issues/80836 from microsoft/tyriar/78574
https://github.com/Microsoft/vscode/commit/6c44ed0e1671832ec4074b35d412550dfe57fef0 Merge pull request https://github.com/microsoft/vscode/issues/79896 from gjsjohnmurray/fix-79618
https://github.com/Microsoft/vscode/commit/ebe12f588977d36739f60b0cf63312aac2e3a508 Editor should become refocused when executing reopen with
https://github.com/Microsoft/vscode/commit/797896d440c03ec0ea1702b744c7bdf8e33fce3a "Reopen with" should close existing editor for resource before opening new editor
https://github.com/Microsoft/vscode/commit/32738944f94c3e237e23beddc1d42d93e797f889 Don\'t rely on WebviewPanelResourceScheme hack for reopen with
https://github.com/Microsoft/vscode/commit/3ce4f1d543cc632c40d300c8204c359f4334cb0a Merge pull request https://github.com/microsoft/vscode/issues/80830 from microsoft/tyriar/80072
https://github.com/Microsoft/vscode/commit/67e37803b7f3f59e945ff61f55d756b9ee28ca62 Add telemtry property for local TS version
https://github.com/Microsoft/vscode/commit/700780d927135cd78c1c653da63b8fd896c72bee Set default colors for \'minimap.selectionHighlight\' in other built in themes
https://github.com/Microsoft/vscode/commit/83e4713fe83a5a7c292ac213dc079cea8afce779 distro
https://github.com/Microsoft/vscode/commit/e1c8327c59ab7e98d3cb2106af5734d18cda21a7 Always use external endpoint on web
https://github.com/Microsoft/vscode/commit/5d828fdcc8e040cac7c71495871638d7f71ebf94 Remove WEBVIEW_ENDPOINT workaround
https://github.com/Microsoft/vscode/commit/54ac29790e6863b8c3d39e6e214e254f57025741 Make sure asWebviewUri also handles replacing commit
https://github.com/Microsoft/vscode/commit/f22a2b814337ed8eb6b91626701ab8f0f966fb45 Update Octicons: sync icon and spin speed
https://github.com/Microsoft/vscode/commit/e4567671d289f798076803830470e10252e137f2 use alt+d instead of alt+D (https://github.com/microsoft/vscode/issues/80824)
https://github.com/Microsoft/vscode/commit/928bafcb493224d69eed31ed21b9566b642270f5 Fix microsoft/vscode-remote-releasehttps://github.com/microsoft/vscode/issues/1353
https://github.com/Microsoft/vscode/commit/e4fc9ba3def4d179004d1c22427b6f34699ba4d7 Try using latest publushed webview iframe version for external webviews
https://github.com/Microsoft/vscode/commit/7a219ab63224565a7ea99bac119fd57c1665a6cf Adding concept of a state for webview editors
https://github.com/Microsoft/vscode/commit/feb3936ff13a154bb17b2a4b856a185ebb71446e :up: distro
https://github.com/Microsoft/vscode/commit/4285736fe83aacbb79f887dcbc20225eb56f3859 debug: support canceling requests
https://github.com/Microsoft/vscode/commit/f6df256c60892c0b31f70a5a872c5d9a1559d126 update - move to browser (https://github.com/microsoft/vscode/issues/80806)
https://github.com/Microsoft/vscode/commit/5e417f3d2299453d817c4edecc8d0644debf38ee calls - fix api command and tests
https://github.com/Microsoft/vscode/commit/5ecf92d64f4927314741988c63f61c1529d101df fix tests, no vscode-dependency please...
https://github.com/Microsoft/vscode/commit/db4102acf82452f4a33e9a5a160703045ee830db Merge pull request https://github.com/microsoft/vscode/issues/80669 from dgozman/for-79228
https://github.com/Microsoft/vscode/commit/f17f3f4cb72d057ab864b2e7dcab132a99bbc644 fix debug UI doesn\'t cleanup spinners on debug session end
https://github.com/Microsoft/vscode/commit/e490d7cf56d21e54ea6ef15bb40f97fe43618818 rawDebugSession implements IDisposable
https://github.com/Microsoft/vscode/commit/96665b6e7aed7465fd2bf0f7585ff3af4b0259c7 calls - add API commands
https://github.com/Microsoft/vscode/commit/ebb728469ac0858c5116847cd2e716ced09f71d6 calls - fix split view issues
https://github.com/Microsoft/vscode/commit/4ac45c1be8e7ac5c36bb8d1317609fbac5792939 :lipstick: named export for extHostProtocol...
https://github.com/Microsoft/vscode/commit/6fc8778fdc6c211bac684c2a22c27979ddb50e5b calls - add CallHierarchyItem to type converter
https://github.com/Microsoft/vscode/commit/debecefce15d087f284db3c3f81c9cf60fad78a3 calls - correct highlights et al
https://github.com/Microsoft/vscode/commit/acfe268248c934f5e18dd6f16a334583b89057d9 calls - more title/message alignments
https://github.com/Microsoft/vscode/commit/3e87b248130a28a0575152e2c23ced24ebadf4ce calls - remember selected direction
https://github.com/Microsoft/vscode/commit/f089ce54337e8768a4b846f311eaec62f1df54f5 calls - reuse change direction action
https://github.com/Microsoft/vscode/commit/ea1bb94d0e1e246bdfc1da8a6ae4242e150f3d4b debt - dispo
https://github.com/Microsoft/vscode/commit/275e6326f9c6b68c8585716f8888d000c9304907 calls - adjust to new root
https://github.com/Microsoft/vscode/commit/ad9950b1154cb72e60cf538834fee44a31c00f07 calls - remove leading dash...
https://github.com/Microsoft/vscode/commit/f1fb95f81041d6cda804fc1037d49782643999e5 calls - tweak title, metatitle of peeks
https://github.com/Microsoft/vscode/commit/27d27b39ada38c5374c1b15956e8d5e7268a45c5 node-debug@1.38.5
https://github.com/Microsoft/vscode/commit/4aa06c261109de43bb313503d789ce92ad130042 calls - remove resolve-function, use document,position tuple when making requests
https://github.com/Microsoft/vscode/commit/16c39d5be4c1d8e9857c31a4bbd7375425321662 update distro
https://github.com/Microsoft/vscode/commit/fc8bedc0fca7a7eb0286df4ff302009b9f1c2c01 DAP add support for breakpoint locations
https://github.com/Microsoft/vscode/commit/8a4bc2268f4fce1ae17519831711b8f6d81b56e3 fixes https://github.com/microsoft/vscode/issues/80780
https://github.com/Microsoft/vscode/commit/b7a5b7cf39f569a570599530ffc855b454e37b06 add webpack-cli as dev dependency for easier testing/running
https://github.com/Microsoft/vscode/commit/9aab002deaca063e9525c4173e36b45edfb26266 push workaround for ts-loader weirdness
https://github.com/Microsoft/vscode/commit/4c45266943228286126e1ad9f840851a0cde594d Merge pull request https://github.com/microsoft/vscode/issues/80703 from microsoft/isidorn/dialogService
https://github.com/Microsoft/vscode/commit/97f6d8386112852709c75312411917214752d33c :up: distro
https://github.com/Microsoft/vscode/commit/153e063486ac96d6b5769483ff0324e64c718353 Fix markdown preview refresh command potentially resetting preview scroll position to top if focused on preview
https://github.com/Microsoft/vscode/commit/8532eb3206a9f6fac43d79bbd37571c8587de137 Remove old-style webview state migration code
https://github.com/Microsoft/vscode/commit/22db92f71b989aa2a2cb8998defbf172bfb2a6cd Add telemetry event when TS server is spawned
https://github.com/Microsoft/vscode/commit/f4524551e98945a3025f6038d3d32500182ff167 Remove gating for TS < 2.2.0 features
https://github.com/Microsoft/vscode/commit/f93f19d23c3e7c71580f0f4336f73cbccf1f94c0 Enable/disable webview find buttons based on if results have been found
https://github.com/Microsoft/vscode/commit/ad5e82042c263d9b79a659a9921f4f73403e6347 Merge pull request https://github.com/microsoft/vscode/issues/79660 from skprabhanjan/fix-79658
https://github.com/Microsoft/vscode/commit/9854a102310f5ab2a6a12018fa51864c8ed7d107 Include user configured custom editors in `open with`
https://github.com/Microsoft/vscode/commit/17e4073398eb85469940854150e3d05614ad5626 More explicit names
https://github.com/Microsoft/vscode/commit/6cd0bf47bb5e29d0489c2b86d211f4fe7056fed3 Add explicit implements WebviewFindDelegate
https://github.com/Microsoft/vscode/commit/a835f40c5b5160a43f247897da2b95a814359b4b Remove deprecated scrollPreviewWithEditorSelection setting
https://github.com/Microsoft/vscode/commit/4dec1f7b496dfd4600006727bacff22ee1ef2bc2 Fix https://github.com/microsoft/vscode/issues/32200
https://github.com/Microsoft/vscode/commit/7720c912d1a34b9ca92d472dfb1516e353e13ece Reduce font-size of new Ocitcons
https://github.com/Microsoft/vscode/commit/37187ba6a2e0f6dd48ca98f895ddaab7b14b4949 Merge pull request https://github.com/microsoft/vscode/issues/80731 from microsoft/terminal_api
https://github.com/Microsoft/vscode/commit/5db460d2fe46251a5c6bdc4796c65b45b7f1eb40 Fix activation of linters on unopened files caused by markdown plugin (https://github.com/microsoft/vscode/issues/80506)
https://github.com/Microsoft/vscode/commit/9083016c89c219b1bad4927485b926c7c50cda9d Remove unused variable
https://github.com/Microsoft/vscode/commit/67dc5341552b4552dfa65245d16b73a629071629 Add selection highlight decorations to minimap, closes https://github.com/microsoft/vscode/issues/21404
https://github.com/Microsoft/vscode/commit/1b9a87375346b38e841260383cc15492081d03ea add cansavecontextkey (https://github.com/microsoft/vscode/issues/80337)
https://github.com/Microsoft/vscode/commit/2afeba767c4f01eeab94dbc3facf8a74bdc32044 Add resourceUriProvider API
https://github.com/Microsoft/vscode/commit/b5b8c4d716358243a27b538b9118aff25e136e52 fixes https://github.com/microsoft/vscode/issues/80452
https://github.com/Microsoft/vscode/commit/beffdb678a7bc43ccd77b2ab2d86e5c15612395b Also use ctrl+enter for multiline search Fix https://github.com/microsoft/vscode/issues/80361
https://github.com/Microsoft/vscode/commit/b035116887d9ddbfc1fc0a8bebda8194824d21b5 jsdoc update
https://github.com/Microsoft/vscode/commit/f5671471838dfc6ec386fc99473653ee0a571b91 Remove unused events
https://github.com/Microsoft/vscode/commit/29fd3ba77d23545410dc8c7f8a88f67227f4ef4e Add sanity test for package.json
https://github.com/Microsoft/vscode/commit/3a384c9c5914d37e860e58438995fd5a5d343810 add test for https://github.com/microsoft/vscode/issues/80688
https://github.com/Microsoft/vscode/commit/fd6bb626e30368630a7b9c914f91298026d90d01 fix tslint warnings...
https://github.com/Microsoft/vscode/commit/c3895965d513eb034927e3586fde53625f30399b :up: distro
https://github.com/Microsoft/vscode/commit/98df43eb9020cef5bc7a480b51795998b14c3c0d Fixes https://github.com/microsoft/vscode/issues/80094: Use US labels for all keys when the keyboard layout does not produce latin characters
https://github.com/Microsoft/vscode/commit/a97b843f2708d91b2f70afa850629aa898d8f249 Merge pull request https://github.com/microsoft/vscode/issues/80713 from microsoft/joh/api/callHierarchy2
https://github.com/Microsoft/vscode/commit/6b8342a7d421265ca122e7a543801413a0ce8f50 fixes https://github.com/microsoft/vscode/issues/80435
https://github.com/Microsoft/vscode/commit/bd2d2a61c1f3ae253eb7c1b60506161a1ce2d70e :up: distro
https://github.com/Microsoft/vscode/commit/eaea43bc8b3e8d0bc78c28efe3d1ece63731c722 :up: distro
https://github.com/Microsoft/vscode/commit/a98ae682dca9588eacc633aaebf030ac528595c9 Have `hasWidgetFocus` return the same underlying value as the event source (https://github.com/microsoft/vscode/issues/71553)
https://github.com/Microsoft/vscode/commit/e229ff62bf67d99edcc62f86432401e4c2fa215d Do not export unused constants
https://github.com/Microsoft/vscode/commit/5efdfac07826029a08942f198ab6b57efebac264 :up: distro
https://github.com/Microsoft/vscode/commit/8d1cac8fc292232c8fa93123d1a93e7f3960baa8 Remove un-used parameter from CustomeExecution callback
https://github.com/Microsoft/vscode/commit/ecc8571a79af049a530190ebeb9c80093b759b2c web - get product config through build and not via API
https://github.com/Microsoft/vscode/commit/ab10e26096a5494b68bc709a405a0dddeb227e0b Fixes https://github.com/microsoft/vscode/issues/80702 (https://github.com/microsoft/vscode/issues/80705)
https://github.com/Microsoft/vscode/commit/d4f1c39dc055fa81dd1ba5eb0bd6b275b9e8de21 update DAP to latest version of cancelation support
https://github.com/Microsoft/vscode/commit/631e8059d6c88d6b6ce1f5e95cfc9f1be007c0b0 fixes https://github.com/microsoft/vscode/issues/80616
https://github.com/Microsoft/vscode/commit/462a8be7d4560770561512eb1109d5ca75e0c8dd Add title API for tree views (https://github.com/microsoft/vscode/issues/80623)
https://github.com/Microsoft/vscode/commit/9aab28daea895051cb24e92ab6026bd233adb43e web - self fullfilling prophecy
https://github.com/Microsoft/vscode/commit/dd7daae7de1d229280400ab9ddc20973fb3178bb associate .i with c
https://github.com/Microsoft/vscode/commit/e4a837a0a327117eaddaf57589caefb98751be21 web - rename entry point
https://github.com/Microsoft/vscode/commit/e75e71f41911633be838344377df26842f2b8c7c debt - :lipstick: product config
https://github.com/Microsoft/vscode/commit/94ee7ce1d70d9990de9cab89b4ea02796e675e0f :up: distro
https://github.com/Microsoft/vscode/commit/211fa02efe8c041fd7baa8ec3dce199d5185aa44 :up: distro
https://github.com/Microsoft/vscode/commit/2d3727b3b19538e149b51d6c3f857636076c1e94 Push `editorResource` concept back into `CustomFileEditorInput`
https://github.com/Microsoft/vscode/commit/6f45980746bee01efc89d66e57b80d2c0fb289b9 Fix normal webview panels being restored as custom editors
https://github.com/Microsoft/vscode/commit/011836a150c5bb55a45ab03896518e159396e031 Prototyping custom editors (https://github.com/microsoft/vscode/issues/77789)
https://github.com/Microsoft/vscode/commit/df802950e0a725a663aa5be9d53a15188c7b2874 xterm@4.0.0
https://github.com/Microsoft/vscode/commit/2fe62e7dfec6fcff6e037e5fbc12082d43d98c0e Use the document that contains the md link as the resource for getting config, not the target document
https://github.com/Microsoft/vscode/commit/28c5988f471996627adfabe31aaa9a679f10dea5 Renaming setting names
https://github.com/Microsoft/vscode/commit/728ee3aa739700785409a25cb9a9c32fd7b43331 Extract
https://github.com/Microsoft/vscode/commit/c307d73664408f51f00fa4aef3a212baa033077e Fix for https://github.com/microsoft/vscode/issues/26659.
(https://github.com/microsoft/vscode/issues/80227)
https://github.com/Microsoft/vscode/commit/db9733d1e3f2ebde16a8b5193d29da0795277cb2 Replace Disposable[] with DisposableStore (https://github.com/microsoft/vscode/issues/80661)
https://github.com/Microsoft/vscode/commit/87cd01498135fa18a71c2a407a1ada3e3a478bb1 Update check, diff add/remove icons, remove SVG font
https://github.com/Microsoft/vscode/commit/8ebd3cea97affac5d85c5e3b587027f9707573da Web: Load onigasm lazy (https://github.com/microsoft/vscode/issues/80642)
https://github.com/Microsoft/vscode/commit/7d8c2f68576a4a0e11c5586df8ea611cf13a19db Fixes for amd resource adoption (https://github.com/microsoft/vscode/issues/80654)
https://github.com/Microsoft/vscode/commit/7bb6df9318dec0b4f616e1c38601903c821ca25e Merge pull request https://github.com/microsoft/vscode/issues/80293 from jasongin/automation
https://github.com/Microsoft/vscode/commit/a556ca989b1935368a5a350ce38c8976ac6f5183 Unset VSCODE_LOGS in code.sh so it writes to the code-oss logs folder and doesn\'t inherit the insiders log folder
https://github.com/Microsoft/vscode/commit/1daa19733ff466b8dfcf7dcba8195d6b58c5edf4 Bump vscode-ripgrep
https://github.com/Microsoft/vscode/commit/44773a13a8860e61b25c0a2bf1f90991323b6ca6 update distro
https://github.com/Microsoft/vscode/commit/cc6aea9d061ceeebb71f2d4acbedbcbc2ca0301a Load script directly
https://github.com/Microsoft/vscode/commit/31b011f8313b19127466fdb238f0d88e13acfa36 Revert "async data tree: use proxies instead"
https://github.com/Microsoft/vscode/commit/7d4a25395f0bdb85ef099e23e9222dd3f8318c42 Have fallback message when code aciton command fails for unknown reason
https://github.com/Microsoft/vscode/commit/b17e5587df7e3c7faf730e29e944820c6123dc34 Surface errors for commands that are part of a code action
https://github.com/Microsoft/vscode/commit/a7704164b4b3a48effe863c908253ae60e6ecfcb Pick up TS 3.6.3 insiders
https://github.com/Microsoft/vscode/commit/1c5b3fd83d3f928bf6c9c574695e53675290a623 Update html service.
Fix https://github.com/microsoft/vscode/issues/80600
https://github.com/Microsoft/vscode/commit/19526f41238259c5db0f182343957b57da14a7c4 snap: add assets verbatim (https://github.com/microsoft/vscode/issues/80585)
https://github.com/Microsoft/vscode/commit/4401cc8350eb0827bb8a3836985120a1ffef5a7a debt - remove a lot of unused icons
https://github.com/Microsoft/vscode/commit/e2c4d07f2b9cdb20d131d3db0a473a5361013a69 update distro
https://github.com/Microsoft/vscode/commit/82a9052ce7c79e9136ac8fa3ae2c901bd4b8e156 more cli args cleanup
https://github.com/Microsoft/vscode/commit/af531331000fbf7137a6d5d40b0ffb4780cf553f Merge pull request https://github.com/microsoft/vscode/issues/79383 from jeanp413/fix-explorer-edit-item
https://github.com/Microsoft/vscode/commit/51912d21c50f11cddf4471021d961fe93e992f4a snippet - honor spread setting for $CLIPBAORD variable with multiple cursors
https://github.com/Microsoft/vscode/commit/93b7ad02df90d65f5b83d189bf51aac704dd635b web - add proper callback landing page
https://github.com/Microsoft/vscode/commit/1ad37cf2c32074b2e7617e637549033f314c8c6f re-enable perf related, hidden arguments
https://github.com/Microsoft/vscode/commit/492def2698487a4c2e315479a216acd001c24060 async data tree: use proxies instead
https://github.com/Microsoft/vscode/commit/5a9913099c077fe0c93abcdb88d0cb67169fefcf undo usage of tsProjectSrc-stream
https://github.com/Microsoft/vscode/commit/c695729eb13edc47787e108b6bd5c4ceaee8d450 remove gulp-tsb hack, fixes https://github.com/microsoft/vscode/issues/80629
https://github.com/Microsoft/vscode/commit/590ccaf26053a8f57a73a6035ad6ebef033cf5d6 Merge pull request https://github.com/microsoft/vscode/issues/80529 from navrkald/prompt_code_inside_wsl
https://github.com/Microsoft/vscode/commit/afffb1e7a77664632e1e6a5079b7e419f015ab1f Contributable variables for configuration resolver (https://github.com/microsoft/vscode/issues/80478)
https://github.com/Microsoft/vscode/commit/9ab449f3bd3921bf0846111980d2f16a4844bf44 use tsb 4.0.2 use project files as src-stream, https://github.com/microsoft/vscode/issues/80632
https://github.com/Microsoft/vscode/commit/96106dbae202708fddbe73db63634cba59def999 update distro
https://github.com/Microsoft/vscode/commit/4e3b4b6356618bc66b0df744a6fa0326d4d80ad9 Merge pull request https://github.com/microsoft/vscode/issues/80422 from dgozman/fix-79196
https://github.com/Microsoft/vscode/commit/81671f3f34d1b3cfdcb8a8a0e266eedb7c057923 fixes https://github.com/microsoft/vscode/issues/80593
https://github.com/Microsoft/vscode/commit/3139e96dd7b90e3b4be3a9a347c1bf1f252e8bf6 update distro
https://github.com/Microsoft/vscode/commit/ca921a50c90681216fa42e80b2e2b2381c176266 Fixes https://github.com/microsoft/vscode/issues/43722: Handle the case where the pasted text ends in \\r\
https://github.com/Microsoft/vscode/commit/227e1a2fa06a31097d00ad5b8d663b72544e08d0 :up: distro
https://github.com/Microsoft/vscode/commit/d0481dca928b7f832eb4c3bfa5898e71f1bb017e web - move selfhost pieces out of workbench
https://github.com/Microsoft/vscode/commit/0030e6ee9250987600837f376dd13bd34fc50d3c :up: terser@4.3.1 (https://github.com/microsoft/vscode/issues/80627)
https://github.com/Microsoft/vscode/commit/b836b675cc4cb8fee7c235150140bdcf5c630cb8 update distro
https://github.com/Microsoft/vscode/commit/7da3d2609bf506f207d8849f574fa9eb6c765ce5 suggest - avoid tokenize when quick suggest options are all on or all off
https://github.com/Microsoft/vscode/commit/ec9acc5ccdf46762c3dcee0847e9bd93bdd67009 Introduce editor.multiCursorPaste (fixes https://github.com/microsoft/vscode/issues/80624)
https://github.com/Microsoft/vscode/commit/3ff6b96ab7bd96e0cd942944ef604837e2b124b5 Update enablement of status bar item in constructor (https://github.com/microsoft/vscode/issues/80517)
https://github.com/Microsoft/vscode/commit/0b3ae7e0898ac4afd39d2463c39ced7c847341e6 Implement URLHandler router which looks at windowId query param (https://github.com/microsoft/vscode/issues/80260)
https://github.com/Microsoft/vscode/commit/51334d5bd9d52c2ad47b2c78c143683544a00933 buildSnapPackage: use the default snapcraft target (https://github.com/microsoft/vscode/issues/80217)
https://github.com/Microsoft/vscode/commit/e4a463507512ff1ba88be399037180d84e0ea17b :lipstick: remove strings.empty
https://github.com/Microsoft/vscode/commit/6ccd016f5648b0347943e2cae15ad89705c0fb62 error handling :lipstick:
https://github.com/Microsoft/vscode/commit/55825e9ad2aa4b4b3f8b2290a54a517bc454a7a5 Fix https://github.com/microsoft/vscode/issues/80536
https://github.com/Microsoft/vscode/commit/d1633405567a53c53056bce9902a537938f83330 Replace Disposable[] with DisposableStore (https://github.com/microsoft/vscode/issues/80447)
https://github.com/Microsoft/vscode/commit/bb021809043a0ccdc1fc9248c12b8dc67e720266 Delay gulp-watch read so watch task isn\'t broken on slower disks.
vscode writes files non-atomically.
This is useful on an azure devbox over ssh.
https://github.com/Microsoft/vscode/commit/0a378fda425b42664ef85b03863da730e15e40c7 Fixes https://github.com/microsoft/vscode/issues/80572: register correct default in JSON schema for enums
https://github.com/Microsoft/vscode/commit/7226e32d1b865b2df4431604be271baef4ffc215 Use shared tsconfig for markdown preview
https://github.com/Microsoft/vscode/commit/3fa22b2f02e6514eeda86f4469fcc923c6ef2065 Use Schemas constants
https://github.com/Microsoft/vscode/commit/d6dab59f161fca3a424871c9b0b73815a8261d11 Allow `file` and `vscode-remote` schemes for links in markdown
https://github.com/Microsoft/vscode/commit/a39d2de39dd5038a1a696800ac9af6dc32a31eab update distro
https://github.com/Microsoft/vscode/commit/a33e5546930a824f155dc0cad3f3de2b108e0538 adopt registerAndGetAmdImageURL
https://github.com/Microsoft/vscode/commit/0041358aee31bdfab44adee3b3531b8f03e3721d Fixes https://github.com/microsoft/vscode/issues/80382
https://github.com/Microsoft/vscode/commit/332c4d9f643c513f65a934271810bc54bd471f10 fix too many updates
https://github.com/Microsoft/vscode/commit/1ba4dc6954da243a087091108e6037f0f7c27ea6 Introduce registerAndGetAmdImageURL and inline such images in the bundle phase in .js
https://github.com/Microsoft/vscode/commit/e05feff0a2e2023a3201bcf687a100b52bf5d9dc xterm@3.15.0-beta108
https://github.com/Microsoft/vscode/commit/42b9564d7c7ea7d9bf8ae0916ccd0080f3077b47 update distro
https://github.com/Microsoft/vscode/commit/09f125d59f6f6e77e532855212d679292b61e95f actually call the `compareByKindAndRank`-function
https://github.com/Microsoft/vscode/commit/e45341c2ee693de08cec9391e7a74cfae36ff83d Merge pull request https://github.com/microsoft/vscode/issues/80560 from microsoft/aeschli/optiondescriptions
https://github.com/Microsoft/vscode/commit/ec9a617c20eec807496678862bb33b036bb7e7db Merge branch \'joao/fix-78388\'
https://github.com/Microsoft/vscode/commit/f8ba99ba5a1cb78302169a0a4cee3ede5b4e8d1b update test file
https://github.com/Microsoft/vscode/commit/b4ead4ed665e1ce2e58d8329c682f78da2d4e89d sortText default value is label, https://github.com/microsoft/vscode/issues/66109
https://github.com/Microsoft/vscode/commit/0f1bd7721b793ed8b7805ba621061ce5a1f5c263 Fix https://github.com/microsoft/vscode/issues/77222
https://github.com/Microsoft/vscode/commit/ea661bb83fa503193c671b63f95764456d806f9f Fix HTML integration test
https://github.com/Microsoft/vscode/commit/f56d927f6e7e41298a7be960060a24442fc1c6bc Restore unit tests
https://github.com/Microsoft/vscode/commit/4f87889a7f6781e2744927ec462276313fec5466 \'u\' flag makes RegExp stricter about which characters can be escaped Fix https://github.com/microsoft/vscode/issues/80455
https://github.com/Microsoft/vscode/commit/bc9a46a307eb77f00b10f20c160410670e184e66 Update Octicons (organization and sync)
https://github.com/Microsoft/vscode/commit/48b30c643f008b94958dacb42525521bcdc42a33 Unify padding across toolbars
https://github.com/Microsoft/vscode/commit/91e2711564742abec933e094743c9b46e9a00cca Do not emit EditorOption in monaco.d.ts
https://github.com/Microsoft/vscode/commit/292685b13fb3c2c75de67b11fe3af0098230c5ce Fix https://github.com/microsoft/vscode/issues/76654 - fix checking that workspace folders exist for searches in remote
https://github.com/Microsoft/vscode/commit/09c580b197c4e7edabe86934372e6325ab95fd55 Add white disconnect icon
https://github.com/Microsoft/vscode/commit/9089a79ecf76a25fccc7c232d80cdd374a7cc66e Refactor editor options (https://github.com/microsoft/vscode/issues/80469)
https://github.com/Microsoft/vscode/commit/9bb27be17bcd190653c1ce1adb39c83069a140d2 Fix https://github.com/microsoft/vscode/issues/79172 - error from telemetry
https://github.com/Microsoft/vscode/commit/f09997e3b3c1a1dd0a709697da12237c177d1ded confine sashes within splitview
https://github.com/Microsoft/vscode/commit/8df7b76ed10d162eb7388dea8fded8ab342f7619 Update distro
https://github.com/Microsoft/vscode/commit/77b3c29911dc62ec7a38c9cc962e09b80131266b xterm@3.15.0-beta107
https://github.com/Microsoft/vscode/commit/8f6a073d1907b7a1d69f34ac6d5b18f16c630f9e Merge pull request https://github.com/microsoft/vscode/issues/80491 from solomatov/expand-shrink-shortcuts
https://github.com/Microsoft/vscode/commit/f8daa9eac4724a6b5b84ab40011a43415c5c3c6b use project-src() to compile what tsconfig-defines
https://github.com/Microsoft/vscode/commit/800fdbc11e3a1ebc82346142c7395f04e6f30fa8 repl: no neet to focusStackFrame when repl expression is added
https://github.com/Microsoft/vscode/commit/2b46fe4c3091a3891eee6e84f90a8cc857ce3c63 adopt gulp-tsb 4.0.0, remove postinstall hacks
https://github.com/Microsoft/vscode/commit/1bbfe023de466dce4c15e79288db375c577899a9 debug service: do not auto focus when a continue is executed
https://github.com/Microsoft/vscode/commit/dd40ab9c75c441fe3ae58887e2f5b9acd5f78084 callstack reduce selection updating
https://github.com/Microsoft/vscode/commit/4d2b972e114e6c59c2caf79e3a017b1412bbc579 debug: reduce call stack flickering if bottom of call stack did not change
https://github.com/Microsoft/vscode/commit/a735b5c9b
'''

In [10]:
text = clean_text(text)

In [12]:
import textsplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=50, length_function=len)
chunks = text_splitter.split_text(text)
chunk_summaries = []
#output_txt_path = os.path.join(output_dir, f'{index + 1}.txt')
for chunk in chunks:
            summary = generate_summary(chunk)
            chunk_summaries.append(summary)
combined_summary = "\n".join(chunk_summaries)
print(combined_summary)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


 The go to definition feature in VS Code 1.38.0 is no longer working as expected. Specifically, when selecting a definition from the peek window and pressing enter, the selection does not change. This issue has been reported before in other issues, but the reporter believes it may be related to recent changes in the tree or workbench tree. To reproduce, disable all extensions and try to go to a definition in the same file. The issue is believed to occur when there are multiple definitions for a method, and the go to definition feature does not work when choosing the definition from the mini popup that appears.

ADDITIONAL INFORMATION:
     - VS Code version: 1.38.0
     - OS version: Ubuntu 16.04 LTS
     - Ruby extension is used for intellisense and go to definitions.
     - The issue occurs when all extensions are disabled, and the reporter is unable to reproduce it.
     - Potential duplicate of issues [go to definition (75978)](httpswww.github.commicrosoftvscodeissues75978) score 0

In [ ]:
import textsplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=50, length_function=len)
df["summary"] = ""

for index, row in tqdm(df.iterrows(), total=len(df), desc="Generating Summaries"):
    wonder_city = row["wonder_city"]
    text_chunk = row["cleaned_information"]
    chunks = text_splitter.split_text(text_chunk)
    chunk_summaries = []

    for chunk in chunks:
        summary = generate_summary(chunk)
        chunk_summaries.append(summary)


    combined_summary = "\n".join(chunk_summaries)
    df.at[index, "summary"] = combined_summary